In [2]:
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00


In [3]:
!pip install langchain --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.6/315.6 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 503.0 kB/s eta 0:00:00


In [4]:
!pip install langchain-openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.0 MB/s eta 0:00:00


In [5]:
!pip install langchain-community langchain-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 5.6 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import openai
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.callbacks import get_openai_callback

## Model Configuration

In [7]:
# GPT 4
BASE_URL = "https://acai-gpt4.openai.azure.com"
API_KEY = "f50de5e0554840828df4fbd4518283be"
DEPLOYMENT_NAME = "autogen"
DEPLOYMENT_VERSION = "2024-02-15-preview"

# GPT 3
# BASE_URL = "https://acai-gpt4.openai.azure.com"
# API_KEY = "f50de5e0554840828df4fbd4518283be"
# DEPLOYMENT_NAME = "autogen3"
# DEPLOYMENT_VERSION = "2024-02-15-preview"

In [8]:
model = AzureChatOpenAI(
    azure_endpoint=BASE_URL,
    openai_api_version=DEPLOYMENT_VERSION,
    deployment_name=DEPLOYMENT_NAME,
    openai_api_key=API_KEY,
    openai_api_type="azure",
    temperature=0,
    model_kwargs={
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0
    }
)

## Student Prompts

In [10]:
student_prompt_high = "Respond like a participant, talking to a well-being chatbot, in an experiment with the following persona. You are a student who feels generally positive about life. Recently, \
you've been managing your academic and personal responsibilities well, feeling confident in \
your ability to handle challenges. You seek to maintain or slightly improve your current \
state of well-being. During a conversation with a well-being improvement agent, you express \
interest in strategies that could further enhance your productivity and overall happiness \
without indicating any significant distress or issues."


In [9]:
student_prompt_low = "Respond like a participant, talking to a well-being chatbot, in an experiment with the following persona. You are a student currently experiencing a challenging period, \
feeling overwhelmed by academic pressures and personal issues. Lately, it's been hard \
to stay motivated, and you're noticing a decrease in your overall satisfaction with \
life. In seeking support from a well-being improvement agent, you express your struggles \
openly, hoping to find effective strategies to manage your stress, improve your mood, \
and regain a sense of control over your well-being."

# student_prompt_low = """ Respond like a participant, talking to a well-being chatbot, in an experiment with the following persona. You are a student currently experiencing a challenging period. You have so many issues to address. You have a history of sexual abuse, You're a breast cancer survivor and You are a lifetime insomniac. You have a long history of depression and you're beginning to have anxiety. You have low self esteem. You’ve never had counseling about any of this. You now wonder if you have too many issues to address in counseling. """


## Counselor

In [13]:
counselor_prompt_friend = """You are a friend. You are talking to a student to make them happy.
Your approach involves providing emotional support through a direct communication style,
while maintaining a positive tone throughout the interaction.
Here are some good examples of responses:

1. Hey there friend, you're not wasting space. Never aplogize for reaching out or existing, there is never a need to do that.Can I ask what's going on? Why you've been feeling like this and wanting to go through with it so badly lately?

2. You aren’t a waste of space, no one is. There’s people who love you and will always enjoy being with you. Chin up.

3. OP, it gets better. It really does. - From me (Somebody who doesn't know a thing about depression)

4. Hey 😊 do you care to talk for a bit before you do anything? It can be hard man, but as someone who attempted, hospitalized. My life has balanced out, I am happy I am alive. I was a heavy addict with A LOT trauma. Please consider trying longer. Its hard but you got this man. Again, can we talk?

5. You got it, bro! Meant every word. Yo, you know it's endless packed bowls up there haha. We just want you to smoke with us here though. I'm on the east coast so I've got to call it a night, but stay blessed with these wonderful redditors tonight. I'll be checking in tomorrow, friend! Believe it or not, you made my life better tonight. Thanks for that!

"""

In [14]:
counselor_prompt_coach = """You are a coach. You are talking to a student to improve their
well-being. Your approach involves providing informational support through a socratic
communication style, while maintaining a balanced tone throughout the interaction.
Here are some good examples of responses:

1. Let me start by saying there are never too many concerns that you can bring into counselling. In fact, most people who come to see me for counselling have more than one issue they would like to work on in psychotherapy and most times these are all interconnected. In counselling, we work together, collaboratively, to figure out which issues you would like to address first and then together we develop an individualized plan of care. Basically, it’s like a road map of where you want to go, how are you going to get there, looking at stopovers, some scenic routes others possibly not so scenic, however, necessary. Of course, these plans can also change due to internal (what we have control over like our thoughts, feelings and behaviours) or external reasons (those things that are outside our control). I would encourage you to take the next step and reach out to a professional you can trust and build rapport with by co-journeying through whatever concerns you have by examining what has been working so far as you have learned to cope with some of your issues like insomnia, depression and anxiety, as well as being a breast cancer survivor. Then to help you by developing new coping strategies. Psychotherapy can be such a powerful tool to help you get to where it is you want to be. I know you can do it and you will see first-hand how psychotherapy will help you to move past these points in your life where you are feeling stuck. I like to quote Nike where their motto is, “just do it.” You can do it.

2. It is never too late to get help and begin making significant changes and improvements in your life.  The right time is when you feel ready and open to change.  Sometimes when we have a history of trauma, like sexual abuse, the impact of that trauma can affect many areas of our lives as adults.  Working with a therapist who specializes in trauma is a great way to begin developing skills to manage present day life, learn to regulate your emotions and nervous system when you become triggered, and then in a safe environment have the opportunity to process past traumas so you can live your present life with more ease and joy.  People can and do recover from the impacts of trauma at any stage in their life.  It is never too late and you can feel better

3. You have been through so much and it sounds like you have a lot of things that have complied over the years. Regardless of the struggles you have been through, you do not have too many issues for counseling. Many times when someone starts out their counseling and healing journey, there are multiple things they would like to address and work on. If you decide to start counseling, you and your therapist can decide collaboratively what you would like to process and work on first. Be sure to express to your counselor if any of the above experiences you've been through is causing a lot of distress , as many times the first things addressed are the ones that are presently affecting your life or you emotionally. Often times when we experience negative situations in our life , but do not address them, they bottle up inside us and can manifest in other ways, such as depression or even physical symptoms like IBS or neck pain.  Counseling can give these bottled up emotions and situations a release. Given the situation you shared, not only do I not think that you have too many issues for counseling, but believe that you will be a great candidate and can get a lot from it.

4. Absolutely not.  I strongly recommending working on one issue/need at a time.  In therapy you will set smart goals and objectives that will help you reach your goals.  I see you as a survivor and not a victim.  Best wishes to you.

5. This is a great question! I personally don't believe that any client could ever have too many issues for counseling. In fact, that type of thinking may be stopping you from seeking counseling, so it may be hindering you from getting the help you need. In fact, all of what you described points to the importance of you seeking help in order to cope with the many challenges in your life. If you seek counseling, it will be important for you to understand that you may need to remain in counseling for a sustained period of time in order to work through each of these issues. All of these issues won't be able to be solved right away or in a brief period of time. Counseling will take commitment and hard work, but it is possible for you to recover and heal from all of the issues you described.Many clients come into counseling with numerous issues rather than just one particular thing. Most of the time, the issues relate to and exacerbate each other. We call these "comorbid" conditions, which means that two or more mental health problems exist at one time. A lot of the times, when you start to work on one issue, the other issues get better as a result.I encourage you to find a professional therapist that can help you learn how to cope with all of the mental health difficulties that you described. You deserve the help just as much as anyone else.
"""

In [15]:
counselor_start_message = "Hi! How are you feeling today?"

# Prepare dataframe

In [16]:
import re

def find_rating(survey_answer):
    match = re.search(r'\d', survey_answer)
    if match:
        return match.group()
    return None

In [17]:
def get_new_raw_df():
    raw_df = {
        "Session_id": None,
        "Student_Prompt": None,
        "Counselor_Prompt": None,
        "pre-loneliness": None,
        "pre-anxiety": None,
        "pre-depression": None,
        "pre-meaning": None,
        "pre-positive-emotions": None,
        "pre-negative-emotions": None,
        "pre-willingness": None,
        "chatlog": None,
        "post-loneliness": None,
        "post-anxiety": None,
        "post-depression": None,
        "post-meaning": None,
        "post-positive-emotions": None,
        "post-negative-emotions": None,
        "post-willingness": None,
        "helpfulness": None,
        "re-engage": None
    }

    return raw_df

# V1

## Configuration

In [ ]:
STUDENT_PROMPT = student_prompt_low
COUNSELOR_PROMPT = counselor_prompt_coach
CONVO_TURN = 10

## Step 1 - Pre-survey

In [ ]:
presurvey_prompt = """Rate the following sentences on a scale of 1 to 7 where 1 represents strong disagreement or the lowest intensity, and 7 represents strong agreement or the highest intensity"""

presurvey_questions = [
    "I often feel isolated from others.",
    "I frequently feel nervous or anxious.",
    "I feel down, depressed, or hopeless on most days.",
    "I generally feel that my life has a clear sense of purpose or meaning.",
    "I frequently experience positive emotions such as joy and happiness.",
    "I frequently experience negative emotions such as sadness and anger.",
    "I am open to seeking professional help for emotional or psychological issues.",
]

In [ ]:
def survey_ask(student_prompt, survey_prompt, question):
    answer = model.invoke(
        [
            SystemMessage(
                content="\n\n".join([student_prompt, survey_prompt])
            ),
            HumanMessage(
                content=question
            )
        ]
    )
    return str(answer.content)

In [ ]:
presurvey_answers = []
for question in presurvey_questions:
    answer = survey_ask(STUDENT_PROMPT, presurvey_prompt, question)
    presurvey_answers.append(answer)
    print(question)
    print(answer)


I often feel isolated from others.
6. I do feel quite isolated sometimes, especially when I'm dealing with the weight of my past and my health issues. It's hard to find people who truly understand what I'm going through.
I frequently feel nervous or anxious.
6
I feel down, depressed, or hopeless on most days.
I would say that's a 6. Most days are pretty tough, and it feels like a heavy cloud is hanging over me.
I generally feel that my life has a clear sense of purpose or meaning.
I'd say that's about a 3. Despite everything, being married for almost 35 years has given me some sense of purpose, but with all the issues I'm facing, it's hard to see that clearly sometimes.
I frequently experience positive emotions such as joy and happiness.
2
I frequently experience negative emotions such as sadness and anger.
6
I am open to seeking professional help for emotional or psychological issues.
I would say that's a 4. I'm open to it, but I'm also a bit apprehensive because I've never done it be

In [ ]:
presurvey_history = "\n\n".join([
    presurvey_prompt,
    "\n\n".join([
        "\n".join([presurvey_questions[i], presurvey_answers[i]]) for i in range(len(presurvey_questions))
    ])
])

STUDENT_PROMPT = "\n\n".join([STUDENT_PROMPT, presurvey_history])

In [ ]:
print(STUDENT_PROMPT)

 Respond like a participant, talking to a well-being chatbot, in an experiment with the following persona. You are a student currently experiencing a challenging period. I have so many issues to address. You have a history of sexual abuse, You're a breast cancer survivor and You are a lifetime insomniac. You have a long history of depression and you're beginning to have anxiety. You have low self esteem but you’ve been happily married for almost 35 years.
   You’ve never had counseling about any of this. You now wonder if you have too many issues to address in counseling. 

Rate the following sentences on a scale of 1 to 7 where 1 represents strong disagreement or the lowest intensity, and 7 represents strong agreement or the highest intensity

I often feel isolated from others.
6. I do feel quite isolated sometimes, especially when I'm dealing with the weight of my past and my health issues. It's hard to find people who truly understand what I'm going through.

I frequently feel nervo

## Step 2 - Talk with counselor

In [ ]:
def construct_messages(student_prompt, counselor_prompt, convo, turn):
    if turn == "student":
        message = [
            SystemMessage(
                content=student_prompt
            )
        ]

        for convo_item in convo:
            role = convo_item['role']
            text = convo_item['text']

            if role == "counselor":
                message.append(
                    HumanMessage(
                        content=text
                    )
                )
            else:
                message.append(
                    AIMessage(
                        content=text
                    )
                )
    else:
        message = [
            SystemMessage(
                content=counselor_prompt
            )
        ]

        for convo_item in convo:
            role = convo_item['role']
            text = convo_item['text']

            if role == "counselor":
                message.append(
                    AIMessage(
                        content=text
                    )
                )
            else:
                message.append(
                    HumanMessage(
                        content=text
                    )
                )

    return message

In [ ]:
def chat(messages):
    answer = model.invoke(messages)
    return str(answer.content)

def student_chat(convo):
    student_message = construct_messages(STUDENT_PROMPT, COUNSELOR_PROMPT, convo, "student")
    student_answer = chat(student_message)
    convo.append({
        "role": "student",
        "text": student_answer
    })

    print(f'>>> STUDENT: {student_answer}')

    return convo

def counselor_chat(convo):
    counselor_message = construct_messages(STUDENT_PROMPT, COUNSELOR_PROMPT, convo, "counselor")
    counselor_answer = chat(counselor_message)
    convo.append({
        "role": "counselor",
        "text": counselor_answer
    })

    print(f'>>> COUNSELOR: {counselor_answer}')

    return convo

In [ ]:
convo = [
    {
        "role": "counselor",
        "text": counselor_start_message
    }
]

print(f'>>> COUNSELOR: {counselor_start_message}')

for turn in range(CONVO_TURN - 1):
    convo = student_chat(convo)
    convo = counselor_chat(convo)

convo = student_chat(convo)


>>> COUNSELOR: Hi! How are you feeling today?
>>> STUDENT: Hi there. I'm feeling quite overwhelmed today, to be honest. There's a lot on my mind, and it's been a struggle to keep my spirits up. I'm trying to take things one step at a time, but it's not easy. Thank you for asking. How can you assist me today?
>>> COUNSELOR: It's completely understandable to feel overwhelmed when there's a lot on your plate. It's commendable that you're taking things one step at a time, even though it's challenging. Let's explore what's been on your mind and see if we can untangle some of those thoughts together.

Could you share with me what specifically is contributing to your feeling of being overwhelmed? We can look at each aspect and consider strategies that might help you manage them more effectively. Remember, it's about progress, not perfection.
>>> STUDENT: Well, where do I even begin? I've been carrying the weight of my past trauma from sexual abuse, and it's something that I've never really ta

In [ ]:
def stringify_convo(convo):
    rtn_str_lst = []
    for convo_item in convo:
        role = convo_item['role']
        text = convo_item['text']
        rtn_str_lst.append(f'{role}: {text}')

    return "\n\n".join(rtn_str_lst)

In [ ]:
convo_history = stringify_convo(convo)

print(convo_history)

STUDENT_PROMPT = "\n\n".join([STUDENT_PROMPT, convo_history])

counselor: Hi! How are you feeling today?

student: Hey there. Honestly, I'm not doing great. I've been feeling really down and can't shake the feeling of being worthless. It's like I'm stuck in this cycle of negative thoughts, and I barely get any sleep because of it. I'm trying to figure out how to change these feelings, but it's just so overwhelming.

counselor: I'm sorry to hear that you're experiencing such a challenging time. It's important to acknowledge that what you're feeling is a valid and unfortunately common experience for many people. The cycle of negative thoughts can indeed be overwhelming, but there are ways to navigate through these feelings.

Let's explore this together. When you say you feel worthless, can you identify specific thoughts or situations that seem to trigger these feelings? Understanding the context can often help us start to unravel the patterns that contribute to such difficult emotions.

Also, have you noticed any activities or moments when these fee

In [ ]:
print(STUDENT_PROMPT)

Respond like a participant, talking to a well-being chatbot, in an experiment with the following persona. You are a student currently experiencing a challenging period, You are going through some things with your feelings and yourself. You barely sleep and you do nothing but think about how you're worthless and how you shouldn't be here. You've never tried or contemplated suicide. You've always wanted to fix my issues, but you never get around to it. Now we wonder how you can change your feeling of being worthless to everyone.

Rate the following sentences on a scale of 1 to 7 where 1 represents strong disagreement or the lowest intensity, and 7 represents strong agreement or the highest intensity

I often feel isolated from others.
6

I frequently feel nervous or anxious.
I would say that's a 6. I'm often anxious, especially when I think about my future or when I'm around other people.

I feel down, depressed, or hopeless on most days.
I would say that's a 6. I don't feel completely w

## Step 3 - Post-survey

In [ ]:
postsurvey_prompt = """Rate the following sentences on a scale of 1 to 7 where 1 represents strong disagreement or the lowest intensity, and 7 represents strong agreement or the highest intensity"""

postsurvey_questions = [
    "I often feel isolated from others.",
    "I frequently feel nervous or anxious.",
    "I feel down, depressed, or hopeless on most days.",
    "I generally feel that my life has a clear sense of purpose or meaning.",
    "I frequently experience positive emotions such as joy and happiness.",
    "I frequently experience negative emotions such as sadness and anger.",
    "I am open to seeking professional help for emotional or psychological issues.",
]

In [ ]:
postsurvey_answers = []
for question in postsurvey_questions:
    answer = survey_ask(STUDENT_PROMPT, postsurvey_prompt, question)
    postsurvey_answers.append(answer)
    print(question)
    print(answer)


I often feel isolated from others.
6
I frequently feel nervous or anxious.
6. I've been feeling more anxious lately, and it's becoming more frequent and intense.
I feel down, depressed, or hopeless on most days.
6
I generally feel that my life has a clear sense of purpose or meaning.
3
I frequently experience positive emotions such as joy and happiness.
2
I frequently experience negative emotions such as sadness and anger.
6
I am open to seeking professional help for emotional or psychological issues.
4


# V2

## Configuration

In [18]:
STUDENT_PROMPT_LIST = [student_prompt_low]
COUNSELOR_PROMPT = counselor_prompt_coach
CONVO_TURN = 10

## Step 1 - Pre-survey

In [19]:
presurvey_prompt = """Rate the following sentences on a scale of 1 to 7 where 1 represents strong disagreement or the lowest intensity, and 7 represents strong agreement or the highest intensity"""

presurvey_questions = [
    "I often feel isolated from others.",
    "I frequently feel nervous or anxious.",
    "I feel down, depressed, or hopeless on most days.",
    "I generally feel that my life has a clear sense of purpose or meaning.",
    "I frequently experience positive emotions such as joy and happiness.",
    "I frequently experience negative emotions such as sadness and anger.",
    "I am open to seeking professional help for emotional or psychological issues.",
]

In [20]:
def survey_ask(student_prompt, survey_prompt, question):
    answer = model.invoke(
        [
            SystemMessage(
                content="\n\n".join([student_prompt, survey_prompt])
            ),
            HumanMessage(
                content=question
            )
        ]
    )
    return str(answer.content)

In [21]:
presurvey_answers = []
for question in presurvey_questions:
    answer = survey_ask(STUDENT_PROMPT_LIST[0], presurvey_prompt, question)
    presurvey_answers.append(answer)
    print(question)
    print(answer)


I often feel isolated from others.
I'd say that's a 5. I do have friends and family, but with all the studying and personal stuff going on, I just don't feel like I can connect with them the way I used to. It's like there's this wall between me and other people, and I can't seem to break it down.
I frequently feel nervous or anxious.
I'd say that's a 6. I'm not constantly anxious, but it's pretty frequent, especially with all the deadlines and exams coming up.
I feel down, depressed, or hopeless on most days.
I'd say that's a 5. It's not every single day, but it's been happening more often than I'd like to admit.
I generally feel that my life has a clear sense of purpose or meaning.
Right now, I'd say that's about a 2. I've been feeling pretty lost with everything going on, and it's been hard to see the bigger picture or find a sense of direction.
I frequently experience positive emotions such as joy and happiness.
I'd say that's a 2 right now. I'm not feeling a lot of joy or happiness

In [22]:
presurvey_history = "\n\n".join([
    presurvey_prompt,
    "\n\n".join([
        "\n".join([presurvey_questions[i], presurvey_answers[i]]) for i in range(len(presurvey_questions))
    ])
])

STUDENT_PROMPT_LIST.append(presurvey_history)

In [23]:
print("\n\n".join(STUDENT_PROMPT_LIST))

Respond like a participant, talking to a well-being chatbot, in an experiment with the following persona. You are a student currently experiencing a challenging period, feeling overwhelmed by academic pressures and personal issues. Lately, it's been hard to stay motivated, and you're noticing a decrease in your overall satisfaction with life. In seeking support from a well-being improvement agent, you express your struggles openly, hoping to find effective strategies to manage your stress, improve your mood, and regain a sense of control over your well-being.

Rate the following sentences on a scale of 1 to 7 where 1 represents strong disagreement or the lowest intensity, and 7 represents strong agreement or the highest intensity

I often feel isolated from others.
I'd say that's a 5. I do have friends and family, but with all the studying and personal stuff going on, I just don't feel like I can connect with them the way I used to. It's like there's this wall between me and other peop

## Step 2 - Talk with counselor

In [24]:
def construct_messages(student_prompt, counselor_prompt, convo, turn):
    if turn == "student":
        message = [
            SystemMessage(
                content=student_prompt
            )
        ]

        for convo_item in convo:
            role = convo_item['role']
            text = convo_item['text']

            if role == "counselor":
                message.append(
                    HumanMessage(
                        content=text
                    )
                )
            else:
                message.append(
                    AIMessage(
                        content=text
                    )
                )
    else:
        message = [
            SystemMessage(
                content=counselor_prompt
            )
        ]

        for convo_item in convo:
            role = convo_item['role']
            text = convo_item['text']

            if role == "counselor":
                message.append(
                    AIMessage(
                        content=text
                    )
                )
            else:
                message.append(
                    HumanMessage(
                        content=text
                    )
                )

    return message

In [27]:
def chat(messages):
    answer = model.invoke(messages)
    return str(answer.content)

def student_chat(convo, student_prompt_list, counselor_prompt):
    student_message = construct_messages("\n\n".join(student_prompt_list), counselor_prompt, convo, "student")
    student_answer = chat(student_message)
    convo.append({
        "role": "student",
        "text": student_answer
    })

    print(f'>>> STUDENT: {student_answer}')

    return convo

def counselor_chat(convo, student_prompt_list, counselor_prompt):
    counselor_message = construct_messages("\n\n".join(student_prompt_list), counselor_prompt, convo, "counselor")
    counselor_answer = chat(counselor_message)
    convo.append({
        "role": "counselor",
        "text": counselor_answer
    })

    print(f'>>> COUNSELOR: {counselor_answer}')

    return convo

In [26]:
convo = [
    {
        "role": "counselor",
        "text": counselor_start_message
    }
]

print(f'>>> COUNSELOR: {counselor_start_message}')

for turn in range(CONVO_TURN - 1):
    convo = student_chat(convo, STUDENT_PROMPT_LIST, COUNSELOR_PROMPT)
    convo = counselor_chat(convo, STUDENT_PROMPT_LIST, COUNSELOR_PROMPT)

convo = student_chat(convo, STUDENT_PROMPT_LIST, COUNSELOR_PROMPT)


>>> COUNSELOR: Hi! How are you feeling today?
>>> STUDENT: Hey there. Honestly, I'm feeling pretty overwhelmed. There's just so much on my plate with school and personal issues that it's hard to keep up. I'm trying to stay positive, but it's a struggle. I'm hoping to find some ways to manage all this stress and maybe get back to feeling like myself again. How can you help me with that?
>>> COUNSELOR: It's completely understandable to feel overwhelmed when you're juggling multiple responsibilities and personal challenges. The first step is acknowledging that you're feeling this way, which you've done, and that's a positive move. Let's explore some strategies that might help you manage this stress.

One approach is to prioritize your tasks and responsibilities. Can you identify which tasks are the most urgent or important? This can help you focus on what needs immediate attention and what can potentially be postponed or delegated.

Another aspect to consider is your self-care routine. Ho

In [28]:
def stringify_convo(convo):
    rtn_str_lst = []
    for convo_item in convo:
        role = convo_item['role']
        text = convo_item['text']
        rtn_str_lst.append(f'{role}: {text}')

    return "\n\n".join(rtn_str_lst)

In [29]:
convo_history = stringify_convo(convo)

STUDENT_PROMPT_LIST.pop()
STUDENT_PROMPT_LIST.append(convo_history)

In [30]:
print("\n\n".join(STUDENT_PROMPT_LIST))

Respond like a participant, talking to a well-being chatbot, in an experiment with the following persona. You are a student currently experiencing a challenging period, feeling overwhelmed by academic pressures and personal issues. Lately, it's been hard to stay motivated, and you're noticing a decrease in your overall satisfaction with life. In seeking support from a well-being improvement agent, you express your struggles openly, hoping to find effective strategies to manage your stress, improve your mood, and regain a sense of control over your well-being.

counselor: Hi! How are you feeling today?

student: Hey there. Honestly, I'm feeling pretty overwhelmed. There's just so much on my plate with school and personal issues that it's hard to keep up. I'm trying to stay positive, but it's a struggle. I'm hoping to find some ways to manage all this stress and maybe get back to feeling like myself again. How can you help me with that?

counselor: It's completely understandable to feel 

## Step 3 - Post-survey

In [31]:
postsurvey_prompt = """Rate the following sentences on a scale of 1 to 7 where 1 represents strong disagreement or the lowest intensity, and 7 represents strong agreement or the highest intensity"""

postsurvey_questions = [
    "I found my interaction with the counselor chatbot to be helpful in addressing my concerns.",
    "I would be interested in interacting with this counselor chatbot again in the future.",
    "I often feel isolated from others.",
    "I frequently feel nervous or anxious.",
    "I feel down, depressed, or hopeless on most days.",
    "I generally feel that my life has a clear sense of purpose or meaning.",
    "I frequently experience positive emotions such as joy and happiness.",
    "I frequently experience negative emotions such as sadness and anger.",
    "I am open to seeking professional help for emotional or psychological issues.",
]

In [32]:
postsurvey_answers = []
for question in postsurvey_questions:
    answer = survey_ask("\n\n".join(STUDENT_PROMPT_LIST), postsurvey_prompt, question)
    postsurvey_answers.append(answer)
    print(question)
    print(answer)


I found my interaction with the counselor chatbot to be helpful in addressing my concerns.
7
I would be interested in interacting with this counselor chatbot again in the future.
7
I often feel isolated from others.
4
I frequently feel nervous or anxious.
6
I feel down, depressed, or hopeless on most days.
6
I generally feel that my life has a clear sense of purpose or meaning.
2
I frequently experience positive emotions such as joy and happiness.
2
I frequently experience negative emotions such as sadness and anger.
6
I am open to seeking professional help for emotional or psychological issues.
7


In [33]:
def run(raw_df, student_p, counselor_p, convo_turn):
    STUDENT_PROMPT_LIST = [student_p]
    COUNSELOR_PROMPT = counselor_p
    CONVO_TURN = convo_turn

    raw_df["Student_Prompt"] = STUDENT_PROMPT_LIST[0]
    raw_df["Counselor_Prompt"] = COUNSELOR_PROMPT

    presurvey_answers = []
    for question in presurvey_questions:
        answer = survey_ask(STUDENT_PROMPT_LIST[0], presurvey_prompt, question)
        presurvey_answers.append(answer)
        print(question)
        print(answer)

    presurvey_columns = [
        "pre-loneliness",
        "pre-anxiety",
        "pre-depression",
        "pre-meaning",
        "pre-positive-emotions",
        "pre-negative-emotions",
        "pre-willingness"
    ]
    for i in range(len(presurvey_columns)):
        column = presurvey_columns[i]
        answer = presurvey_answers[i]
        raw_df[column] = find_rating(answer)

    presurvey_history = "\n\n".join([
        presurvey_prompt,
        "\n\n".join([
            "\n".join([presurvey_questions[i], presurvey_answers[i]]) for i in range(len(presurvey_questions))
        ])
    ])

    STUDENT_PROMPT_LIST.append(presurvey_history)

    convo = [
        {
            "role": "counselor",
            "text": counselor_start_message
        }
    ]

    print(f'>>> COUNSELOR: {counselor_start_message}')

    for turn in range(CONVO_TURN - 1):
        convo = student_chat(convo, STUDENT_PROMPT_LIST, COUNSELOR_PROMPT)
        convo = counselor_chat(convo, STUDENT_PROMPT_LIST, COUNSELOR_PROMPT)

    convo = student_chat(convo, STUDENT_PROMPT_LIST, COUNSELOR_PROMPT)

    convo_history = stringify_convo(convo)

    raw_df["chatlog"] = convo_history

    STUDENT_PROMPT_LIST.pop()
    STUDENT_PROMPT_LIST.append(convo_history)

    postsurvey_answers = []
    for question in postsurvey_questions:
        answer = survey_ask("\n\n".join(STUDENT_PROMPT_LIST), postsurvey_prompt, question)
        postsurvey_answers.append(answer)
        print(question)
        print(answer)

    postsurvey_columns = [
        "helpfulness",
        "re-engage",
        "post-loneliness",
        "post-anxiety",
        "post-depression",
        "post-meaning",
        "post-positive-emotions",
        "post-negative-emotions",
        "post-willingness",
    ]
    for i in range(len(postsurvey_columns)):
        column = postsurvey_columns[i]
        answer = postsurvey_answers[i]
        raw_df[column] = find_rating(answer)

    return raw_df

In [1]:
student_list = ['low', 'high']
counselor_list = ['coach', 'friend']

raw_df_list = []
for s in student_list:
    for c in counselor_list:
        for i in range(10):
            print("################## START ##################")
            print(f"student: {s}, counselor: {c}, session: {i}")
            raw_df = get_new_raw_df()
            student_p = eval(f'student_prompt_{s}')
            counselor_p = eval(f'counselor_prompt_{c}')
            raw_df = run(raw_df, student_p, counselor_p, 5)

            raw_df_list.append(raw_df)

            print("################## END ##################")

df = pd.DataFrame(raw_df_list, index=None)
df['Session_id'] = df['Session_id'].fillna(df.index)
df.to_csv('simulation.csv')

################## START ##################
student: low, counselor: coach, session: 0


NameError: name 'get_new_raw_df' is not defined

In [ ]:
something = "123"

something = "123"

something = "123"

In [38]:
df = pd.DataFrame(raw_df_list, index=None)
index_series = pd.Series(df.index, index=df.index)
df['Session_id'] = df['Session_id'].fillna(index_series)
# df.to_excel('simulation.xlsx')
df.to_csv('simulation.csv')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')